### If import shows error then install these files first

In [ ]:
pip install openpyxl

### Import all the necessary libraries

In [ ]:
import os
import pandas as pd 
import requests
import json
import shutil
import time # Not so relevent but yes its important 
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
from openpyxl.styles import Alignment

### Dataset set downloading model for a dynamic javascript interface

In [ ]:

# write the base api url for the request site for the datasets
base_url = "https://indiawris.gov.in/Dataset/Ground%20Water%20Level"

# All states are given in a list form 
state_districts = {
    "Andhra Pradesh": ["Anantapur", "Chittoor", "East Godavari", "Guntur", "Kadapa", "Krishna", "Kurnool", "Nellore", "Prakasam", "Srikakulam", "Visakhapatnam", "Vizianagaram", "West Godavari"],

    "Arunachal Pradesh": ["Anjaw", "Changlang", "Dibang Valley", "East Kameng", "East Siang", "Kamle", "Kra Daadi", "Kurung Kumey", "Lohit", "Longding", "Lower Dibang Valley", "Lower Siang", "Lower Subansiri", "Namsai", "Pakke Kessang", "Papum Pare", "Shi Yomi", "Siang", "Tawang", "Tirap", "Upper Siang", "Upper Subansiri", "West Kameng", "West Siang"],

    "Assam": ["Baksa", "Barpeta", "Biswanath", "Bongaigaon", "Cachar", "Charaideo", "Chirang", "Darrang", "Dhemaji", "Dhubri", "Dibrugarh", "Dima Hasao", "Goalpara", "Golaghat", "Hailakandi", "Hojai", "Jorhat", "Kamrup", "Kamrup Metropolitan", "Karbi Anglong", "Karimganj", "Kokrajhar", "Lakhimpur", "Majuli", "Morigaon", "Nagaon", "Nalbari", "Sivasagar", "Sonitpur", "South Salmara-Mankachar", "Tinsukia", "Udalguri", "West Karbi Anglong"],

    "Bihar": ["Araria", "Arwal", "Aurangabad", "Banka", "Begusarai", "Bhagalpur", "Bhojpur", "Buxar", "Darbhanga", "East Champaran", "Gaya", "Gopalganj", "Jamui", "Jehanabad", "Kaimur", "Katihar", "Khagaria", "Kishanganj", "Lakhisarai", "Madhepura", "Madhubani", "Munger", "Muzaffarpur", "Nalanda", "Nawada", "Patna", "Purnia", "Rohtas", "Saharsa", "Samastipur", "Saran", "Sheikhpura", "Sheohar", "Sitamarhi", "Siwan", "Supaul", "Vaishali", "West Champaran"],

    "Chhattisgarh": ["Balod", "Baloda Bazar", "Balrampur", "Bastar", "Bemetara", "Bijapur", "Bilaspur", "Dantewada", "Dhamtari", "Durg", "Gariaband", "Gaurela-Pendra-Marwahi", "Janjgir-Champa", "Jashpur", "Kabirdham", "Kanker", "Kondagaon", "Korba", "Koriya", "Mahasamund", "Mungeli", "Narayanpur", "Raigarh", "Raipur", "Rajnandgaon", "Sukma", "Surajpur", "Surguja"],

    "Goa": ["North Goa", "South Goa"],

    "Gujarat": ["Ahmedabad", "Amreli", "Anand", "Aravalli", "Banaskantha", "Bharuch", "Bhavnagar", "Botad", "Chhota Udaipur", "Dahod", "Dang", "Devbhoomi Dwarka", "Gandhinagar", "Gir Somnath", "Jamnagar", "Junagadh", "Kheda", "Kutch", "Mahisagar", "Mehsana", "Morbi", "Narmada", "Navsari", "Panchmahal", "Patan", "Porbandar", "Rajkot", "Sabarkantha", "Surat", "Surendranagar", "Tapi", "Vadodara", "Valsad"],

    "Haryana": ["Ambala", "Bhiwani", "Charkhi Dadri", "Faridabad", "Fatehabad", "Gurugram", "Hisar", "Jhajjar", "Jind", "Kaithal", "Karnal", "Kurukshetra", "Mahendragarh", "Nuh", "Palwal", "Panchkula", "Panipat", "Rewari", "Rohtak", "Sirsa", "Sonipat", "Yamunanagar"],

    "Himachal Pradesh": ["Bilaspur", "Chamba", "Hamirpur", "Kangra", "Kinnaur", "Kullu", "Lahaul and Spiti", "Mandi", "Shimla", "Sirmaur", "Solan", "Una"],

    "Jharkhand": ["Bokaro", "Chatra", "Deoghar", "Dhanbad", "Dumka", "East Singhbhum", "Garhwa", "Giridih", "Godda", "Gumla", "Hazaribagh", "Jamtara", "Khunti", "Koderma", "Latehar", "Lohardaga", "Pakur", "Palamu", "Ramgarh", "Ranchi", "Sahibganj", "Seraikela Kharsawan", "Simdega", "West Singhbhum"],

    "Karnataka": ["Bagalkot", "Ballari", "Belagavi", "Bengaluru Rural", "Bengaluru Urban", "Bidar", "Chamarajanagar", "Chikkaballapur", "Chikkamagaluru", "Chitradurga", "Dakshina Kannada", "Davanagere", "Dharwad", "Gadag", "Hassan", "Haveri", "Kalaburagi", "Kodagu", "Kolar", "Koppal", "Mandya", "Mysuru", "Raichur", "Ramanagara", "Shivamogga", "Tumakuru", "Udupi", "Uttara Kannada", "Vijayapura", "Yadgir"],

    "Kerala": ["Alappuzha", "Ernakulam", "Idukki", "Kannur", "Kasaragod", "Kollam", "Kottayam", "Kozhikode", "Malappuram", "Palakkad", "Pathanamthitta", "Thiruvananthapuram", "Thrissur", "Wayanad"],

    "Madhya Pradesh": ["Agar Malwa", "Alirajpur", "Anuppur", "Ashoknagar", "Balaghat", "Barwani", "Betul", "Bhind", "Bhopal", "Burhanpur", "Chhatarpur", "Chhindwara", "Damoh", "Datia", "Dewas", "Dhar", "Dindori", "Guna", "Gwalior", "Harda", "Hoshangabad", "Indore", "Jabalpur", "Jhabua", "Katni", "Khandwa", "Khargone", "Mandla", "Mandsaur", "Morena", "Narsinghpur", "Neemuch", "Panna", "Raisen", "Rajgarh", "Ratlam", "Rewa", "Sagar", "Satna", "Sehore", "Seoni", "Shahdol", "Shajapur", "Sheopur", "Shivpuri", "Sidhi", "Singrauli", "Tikamgarh", "Ujjain", "Umaria", "Vidisha"],

    "Maharashtra": ["Ahmednagar", "Akola", "Amravati", "Aurangabad", "Beed", "Bhandara", "Buldhana", "Chandrapur", "Dhule", "Gadchiroli", "Gondia", "Hingoli", "Jalgaon", "Jalna", "Kolhapur", "Latur", "Mumbai City", "Mumbai Suburban", "Nagpur", "Nanded", "Nandurbar", "Nashik", "Osmanabad", "Palghar", "Parbhani", "Pune", "Raigad", "Ratnagiri", "Sangli", "Satara", "Sindhudurg", "Solapur", "Thane", "Wardha", "Washim", "Yavatmal"],

    "Manipur": ["Bishnupur", "Chandel", "Churachandpur", "Imphal East", "Imphal West", "Jiribam", "Kakching", "Kamjong", "Kangpokpi", "Noney", "Pherzawl", "Senapati", "Tamenglong", "Tengnoupal", "Thoubal", "Ukhrul"],

    "Meghalaya": ["East Garo Hills", "East Jaintia Hills", "East Khasi Hills", "North Garo Hills", "Ri-Bhoi", "South Garo Hills", "South West Garo Hills", "South West Khasi Hills", "West Garo Hills", "West Jaintia Hills", "West Khasi Hills"],

    "Mizoram": ["Aizawl", "Champhai", "Hnahthial", "Khawzawl", "Kolasib", "Lawngtlai", "Lunglei", "Mamit", "Saiha", "Serchhip", "Saitual"],

    "Nagaland": ["Dimapur", "Kiphire", "Kohima", "Longleng", "Mokokchung", "Mon", "Noklak", "Peren", "Phek", "Tuensang", "Wokha", "Zunheboto"],

    "Odisha": ["Angul", "Balangir", "Balasore", "Bargarh", "Bhadrak", "Boudh", "Cuttack", "Deogarh", "Dhenkanal", "Gajapati", "Ganjam", "Jagatsinghpur", "Jajpur", "Jharsuguda", "Kalahandi", "Kandhamal", "Kendrapara", "Kendujhar", "Khordha", "Koraput", "Malkangiri", "Mayurbhanj", "Nabarangpur", "Nayagarh", "Nuapada", "Puri", "Rayagada", "Sambalpur", "Subarnapur", "Sundargarh"],

    "Punjab": ["Amritsar", "Barnala", "Bathinda", "Faridkot", "Fatehgarh Sahib", "Fazilka", "Ferozepur", "Gurdaspur", "Hoshiarpur", "Jalandhar", "Kapurthala", "Ludhiana", "Mansa", "Moga", "Muktsar", "Pathankot", "Patiala", "Rupnagar", "Sahibzada Ajit Singh Nagar", "Sangrur", "Shahid Bhagat Singh Nagar", "Tarn Taran"],

    "Rajasthan": ["Ajmer", "Alwar", "Banswara", "Baran", "Barmer", "Bharatpur", "Bhilwara", "Bikaner", "Bundi", "Chittorgarh", "Churu", "Dausa", "Dholpur", "Dungarpur", "Hanumangarh", "Jaipur", "Jaisalmer", "Jalore", "Jhalawar", "Jhunjhunu", "Jodhpur", "Karauli", "Kota", "Nagaur", "Pali", "Pratapgarh", "Rajsamand", "Sawai Madhopur", "Sikar", "Sirohi"],
    "Sikkim": ["East Sikkim", "North Sikkim", "South Sikkim", "West Sikkim"],

    "Tamil Nadu": ["Ariyalur", "Chengalpattu", "Chennai", "Coimbatore", "Cuddalore", "Dharmapuri", "Dindigul", "Erode", "Kallakurichi", "Kanchipuram", "Kanyakumari", "Karur", "Krishnagiri", "Madurai", "Nagapattinam", "Namakkal", "Nilgiris", "Perambalur", "Pudukkottai", "Ramanathapuram", "Ranipet", "Salem", "Sivaganga", "Tenkasi", "Thanjavur", "Theni", "Thoothukudi", "Tiruchirappalli", "Tirunelveli", "Tiruppur", "Tiruvallur", "Tiruvannamalai", "Tiruvarur", "Vellore", "Viluppuram", "Virudhunagar"],

    "Telangana": ["Adilabad", "Bhadradri Kothagudem", "Hyderabad", "Jagtial", "Jangaon", "Jayashankar Bhoopalpally", "Jogulamba Gadwal", "Kamareddy", "Karimnagar", "Khammam", "Komaram Bheem Asifabad", "Mahabubabad", "Mahabubnagar", "Mancherial", "Medak", "Medchal-Malkajgiri", "Mulugu", "Nagarkurnool", "Nalgonda", "Narayanpet", "Nirmal", "Nizamabad", "Peddapalli", "Rajanna Sircilla", "Rangareddy", "Sangareddy", "Siddipet", "Suryapet", "Vikarabad", "Wanaparthy", "Warangal (Rural)", "Warangal (Urban)", "Yadadri Bhuvanagiri"],

    "Tripura": ["Dhalai", "Gomati", "Khowai", "North Tripura", "Sepahijala", "South Tripura", "Unakoti", "West Tripura"],

    "Uttar Pradesh": ["Agra", "Aligarh", "Allahabad (Prayagraj)", "Ambedkar Nagar", "Amethi", "Amroha", "Auraiya", "Azamgarh", "Baghpat", "Bahraich", "Ballia", "Balrampur", "Banda", "Barabanki", "Bareilly", "Basti", "Bhadohi", "Bijnor", "Bulandshahr", "Chandauli", "Chitrakoot", "Deoria", "Etah", "Etawah", "Faizabad", "Farrukhabad", "Fatehpur", "Firozabad", "Gautam Buddha Nagar", "Ghaziabad", "Ghazipur", "Gonda", "Gorakhpur", "Hamirpur", "Hapur", "Hardoi", "Hathras", "Jalaun", "Jaunpur", "Jhansi", "Kannauj", "Kanpur Dehat", "Kanpur Nagar", "Kasganj", "Kaushambi", "Kushinagar", "Lakhimpur Kheri", "Lalitpur", "Lucknow", "Maharajganj", "Mahoba", "Mainpuri", "Mathura", "Mau", "Meerut", "Mirzapur", "Moradabad", "Muzaffarnagar", "Pilibhit", "Pratapgarh", "Rae Bareli", "Rampur", "Saharanpur", "Sambhal", "Sant Kabir Nagar", "Shahjahanpur", "Shamli", "Shravasti", "Siddharthnagar", "Sitapur", "Sonbhadra", "Sultanpur", "Unnao", "Varanasi"],

    "Uttarakhand": ["Almora", "Bageshwar", "Chamoli", "Champawat", "Dehradun", "Haridwar", "Nainital", "Pauri Garhwal", "Pithoragarh", "Rudraprayag", "Tehri Garhwal", "Udham Singh Nagar", "Uttarkashi"],

    "West Bengal": ["Alipurduar", "Bankura", "Birbhum", "Cooch Behar", "Dakshin Dinajpur", "Darjeeling", "Hooghly", "Howrah", "Jalpaiguri", "Jhargram", "Kalimpong", "Kolkata", "Malda", "Murshidabad", "Nadia", "North 24 Parganas", "Paschim Bardhaman", "Paschim Medinipur", "Purba Bardhaman", "Purba Medinipur", "Purulia", "South 24 Parganas", "Uttar Dinajpur"],

    # Union Territories:

    "Andaman and Nicobar Islands": ["Nicobar", "North and Middle Andaman", "South Andaman"],

    "Chandigarh": ["Chandigarh"],

    "Dadra and Nagar Haveli and Daman and Diu": ["Dadra and Nagar Haveli", "Daman", "Diu"],

    "Delhi": ["Central Delhi", "East Delhi", "New Delhi", "North Delhi", "North East Delhi", "North West Delhi", "Shahdara", "South Delhi", "South East Delhi", "South West Delhi", "West Delhi"],

    "Jammu and Kashmir": ["Anantnag", "Bandipora", "Baramulla", "Budgam", "Doda", "Ganderbal", "Jammu", "Kathua", "Kishtwar", "Kulgam", "Kupwara", "Poonch", "Pulwama", "Rajouri", "Ramban", "Reasi", "Samba", "Shopian", "Srinagar", "Udhampur"],

    "Ladakh": ["Kargil", "Leh"],

    "Lakshadweep": ["Lakshadweep"],

    "Puducherry": ["Karaikal", "Mahe", "Puducherry", "Yanam"]
}

# Basic feild requirements for the site
agency_name = "CGWB"
start_year = 1993
end_year = 2024
base_dir = "D:\\Actual_dataset" #Base Directory of the file where the folder created will be saved 

#loop to express each label as separate will take a lot of time 
for year in range(start_year , end_year ):
    year += 1
    start_date = f"{year}-01-01"
    end_date = f"{year}-12-31"
    print(f"Start: {start_date}, End: {end_date}")
    
    # Internal loop to state the params to the server
    for state, districts in state_districts.items():
        for district in districts:
            params = {
                "stateName": state,
                "districtName": district,
                "agencyName": agency_name, 
                "startdate": start_date,
                "enddate": end_date,
                "download": "false",
                "page": 0,
                "size": 1000
            }
            #trying each possible request process
            try:
                response = requests.post(base_url, params=params)
                response.raise_for_status()
                data = response.json()

                # for successful request saving it somewhere 
                for i in range(start_year + 1, end_year):
                    folder_name = f"data_json_{year}"
                    folder_path = os.path.join(base_dir, folder_name)
                    os.makedirs(folder_path, exist_ok=True)
                    print(f"[SUCCESS] {state} - {district}: {len(data)} records")
            
                filename = f"{state}_{district}_groundwater.json".replace(" ", "_")
                file_path = os.path.join(folder_path, filename)
                with open(file_path, "w") as f:
                    json.dump(data, f, indent=4)
            except requests.exceptions.RequestException as e:
                print(f"[ERROR] {state} - {district}: {str(e)}")
    
            time.sleep(1) # Sleep or pause time before another request so that server will not crash 
    
    
    # converting the saved JSON file to .csv files simuntaneosly
    input_dir = folder_path
    output_dir = os.path.join(base_dir, f"data_csv_{year}")
    os.makedirs(output_dir, exist_ok=True)

    for filename in os.listdir(input_dir):
        if filename.endswith(".json"):
            input_path = os.path.join(input_dir, filename)
            try:
                with open(input_path, "r", encoding="utf-8") as file:
                    data = json.load(file)

                df = None
                if isinstance(data, dict):
                    for key in data:
                        if isinstance(data[key], list):
                            df = pd.DataFrame(data[key])
                            break
                elif isinstance(data, list):
                    df = pd.DataFrame(data)mn

                if df is not None and not df.empty:
                    csv_filename = os.path.splitext(filename)[0] + ".csv"
                    csv_path = os.path.join(output_dir, csv_filename)
                    df.to_csv(csv_path, index=False)
                    print(f"Converted {filename} to CSV")
                else:
                    print(f"No data found in {filename}, skipping")

            except Exception as e:
                print(f"Failed to convert {filename}: {e}")

### Combining all the obtained .csv files into one file of the respective year

In [ ]:
# Year range and base directory
startyear = 1993
endyear = 2024
base_dir = "D:\\Actual_dataset\\data_csv"

# Loop over each year
for year in range(startyear, endyear + 1):
    folder_name = f"data_csv_{year}"
    folder_path = os.path.join(base_dir, folder_name)
    output_file = os.path.join(base_dir, f"combined_files_{year}.csv")

    # List to hold all DataFrames for this year
    dfs = []

    if not os.path.exists(folder_path):
        print(f"Folder not found: {folder_path}")
        continue

    # Loop through files in that year's folder
    for filename in os.listdir(folder_path):
        if filename.endswith(".csv"):
            file_path = os.path.join(folder_path, filename)
            try:
                df = pd.read_csv(file_path)
                df['Source_File'] = filename  # Optional: track source
                dfs.append(df)
                print(f"[{year}] Loaded: {filename}")
            except Exception as e:
                print(f"Failed to read {filename}: {e}")

    # Combine and save the CSVs for this year
    if dfs:
        combined_df = pd.concat(dfs, ignore_index=True)
        combined_df.to_csv(output_file, index=False)
        print(f"Saved combined CSV for {year} to: {output_file}")
    else:
        print(f"No CSVs loaded for {year}")

### Combining all the combined .csv file into large .csv file to easily interpret the data
**Note** - Very large files may crash the memory the os you are working on while opening be cautious

In [ ]:
startyear = 1995
endyear = 2024
base_dir = "D:\\Actual_dataset\\data_csv"

# Loop over each year
for year in range(startyear, endyear + 1):
    folder_name = f"combined_files"
    folder_path = os.path.join(base_dir, folder_name)
    output_file = os.path.join(base_dir, f"mega_combined_data_csv.csv")

    # List to hold all DataFrames for this year
    dfs = []

    if not os.path.exists(folder_path):
        print(f"Folder not found: {folder_path}")
        continue

    # Loop through files in that year's folder
    for filename in os.listdir(folder_path):
        if filename.endswith(".csv"):
            file_path = os.path.join(folder_path, filename)
            try:
                df = pd.read_csv(file_path)
                df['Source_File'] = filename  # Optional: track source
                dfs.append(df)
                print(f"[{year}] Loaded: {filename}")
            except Exception as e:
                print(f"Failed to read {filename}: {e}")

    # Combine and save the CSVs for this year
    if dfs:
        combined_df = pd.concat(dfs, ignore_index=True)
        combined_df.to_csv(output_file, index=False)
        print(f"Saved combined CSV for {year} to: {output_file}")
    else:
        print(f"No CSVs loaded for {year}")

### Modifying the table to our desired format for easy assessment 
* all the required or unique fields like vilage, station code... should be on the column 
* all the data values should be on the row.

In [ ]:
#Load data
df = pd.read_csv("D:\\Actual_dataset\\data_csv\\mega_combined_data_csv_CGWB.csv", parse_dates=["dataTime"])

# Extract datetime components
df["year"] = df["dataTime"].dt.year
df["month_num"] = df["dataTime"].dt.month
df["month_name"] = df["dataTime"].dt.strftime("%b")
df["time"] = df["dataTime"].dt.strftime("%H:%M")

# ID columns
id_columns = [
    "state", "district", "tehsil", "block", "village",
    "latitude", "longitude", 
    "stationCode", "stationName", "stationType", 
    "wellType", "wellAquiferType", "wellDepth" 
]

# Melt
melted = df.melt(
    id_vars=id_columns + ["year", "month_num", "month_name", "time"],
    value_vars=["dataValue"],
    var_name="variable",
    value_name="value"
)

# MultiIndex for pivot: (year, "Month Time")
melted["col_header"] = list(zip(
    melted["year"],
    melted["month_num"],
    melted["month_name"] + " " + melted["time"]
))

# Pivot
pivoted = melted.pivot_table(
    index=id_columns,
    columns="col_header",
    values="value",
    aggfunc="max"
)

# Sort columns: by year, then month_num
pivoted.columns = pd.MultiIndex.from_tuples(pivoted.columns)
pivoted = pivoted.sort_index(axis=1, level=[0, 1])

# Drop month_num from index (keep only year and "Month Time")
pivoted.columns = pd.MultiIndex.from_tuples([
    (year, label) for (year, month_num, label) in pivoted.columns
])

# Reset index to make it saveable
pivoted = pivoted.reset_index()

# Save to Excel to preserve multi-index
pivoted.to_excel("reshaped_calendar_multilevel.xlsx", index=True)
print("Saved to 'reshaped_calendar_multilevel.xlsx' with calendar-style column headers.")
